In [1]:
import nltk
import pandas as pd

In [2]:
df=pd.read_csv('Twitter_Data.csv', encoding = "ISO-8859-1",usecols=['Sentiment','Tweet'])

In [3]:
df.shape

(1048575, 2)

## My machine being slow, I am gonna use only 2k reviews out of 1m

In [4]:
pos_tweets=df[800000:801000]  
neg_tweets=df[:1000]

pos_tweets=pos_tweets.reset_index()

pos_tweets=pos_tweets[['Sentiment','Tweet']]

In [5]:
new_df=pd.concat([neg_tweets,pos_tweets])

new_df=new_df.reset_index()
new_df=new_df[['Sentiment','Tweet']]

new_df=new_df.replace(to_replace=0,value='negative')
new_df=new_df.replace(to_replace=4,value='positive')
new_df

,Sentiment,Tweet
0,negative,is upset that he can't update his Facebook by ...
1,negative,@Kenichan I dived many times for the ball. Man...
2,negative,my whole body feels itchy and like its on fire
3,negative,"@nationwideclass no, it's not behaving at all...."
4,negative,@Kwesidei not the whole crew
5,negative,Need a hug
6,negative,@LOLTrish hey long time no see! Yes.. Rains a...
7,negative,@Tatiana_K nope they didn't have it
8,negative,@twittera que me muera ?
9,negative,spring break in plain city... it's snowing


In [6]:
tl=[]
for i in range(len(new_df)):
    tl.append((df['Tweet'][i],df['Sentiment'][i]))

In [7]:
tweets = []
for (words, sentiment) in zip(new_df['Tweet'],new_df['Sentiment']):
    words_filtered = [e.lower() for e in words.split() if len(e) >= 3 and sentiment] 
    tweets.append((words_filtered, sentiment))

In [8]:
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    print(wordlist)
    word_features = wordlist.keys()
    return wordlist

In [9]:
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

In [10]:
word_features = get_word_features(get_words_in_tweets(tweets))
word_features=word_features.keys()

<FreqDist with 7391 samples and 20709 outcomes>


In [11]:
def extract_features(document):
    document_words = set(document)
    features = {}
    l=[]
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
        l.append(word)
    return features

In [12]:
training_set = nltk.classify.apply_features(extract_features, tweets)


In [13]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [14]:
test_df=pd.DataFrame.sample(new_df,n=1000)

In [15]:
yp=[]
for i in test_df['Tweet']:
    yp.append(classifier.classify(extract_features(i.split())))

y=test_df['Sentiment']

In [16]:
from sklearn.metrics import accuracy_score

In [17]:
accuracy_score(y,yp)

0.964

# Using Tweet_miner

In [25]:
from TweetMiner import mineTweets

In [26]:
tweetList=mineTweets('realdonaldtrump')

In [27]:
pred=[]
for i in range(len(tweetList)):
    pred.append(classifier.classify(extract_features(tweetList[i].split())))

In [28]:
tweetSentiment=pd.DataFrame(data=[pred,tweetList]).T

In [29]:
tweetSentiment.columns=['Sentiment','Tweet']

In [30]:
posT=0
negT=0
for i in range(len(tweetSentiment)):
    if tweetSentiment.Sentiment[i]=='positive':
        posT+=1
    else:
        negT+=1

In [31]:
tweetSentiment

,Sentiment,Tweet
0,negative,"Go back to bed, traitor. We got this. The last..."
1,negative,"Dude, I wouldn’t be bragging about the well ru..."
2,positive,California is an amazing state. Were the 6th l...
3,positive,*5th
4,positive,An upgrade! Even better
5,negative,"5th Largest WORLD economy, 30 Billion Surplus...."
6,negative,Fvckin A folks. If it weren’t for California t...
7,negative,"Better idea, let’s cut all state funding to th..."
8,negative,We need gun control now. Stop letting American...
9,positive,Trump spends more time attacking the Mayor of ...


In [24]:
print('Total number of Tweets:',len(tweetSentiment))
print(f"Positive: {posT}\tNegative: {negT}")
print("Positive to Negative ratio: ", posT/negT)

Total number of Tweets: 270
Positive: 144	Negative: 126
Positive to Negative ratio:  1.1428571428571428
